# Table of Contents
* [Learning Objectives:](#Learning-Objectives:)
* [Linear algebra](#Linear-algebra)
	* [Linear equation systems](#Linear-equation-systems)
	* [Eigenvalues and eigenvectors](#Eigenvalues-and-eigenvectors)
	* [Matrix operations](#Matrix-operations)
		* [A matrix algebra example with feed forward/backward chemical reactors](#A-matrix-algebra-example-with-feed-forward/backward-chemical-reactors)
	* [Sparse matrices](#Sparse-matrices)


# Learning Objectives:

After completion of this module, learners should be able to:

* explain and use standard Python numerical functions for dense and sparse linear algebra

# Linear algebra

The linear algebra module contains a lot of matrix-related functions, including linear equation solving, eigenvalue solvers, matrix functions (for example matrix-exponentiation), a number of different decompositions (SVD, LU, cholesky), etc. 

Detailed documetation is available at: http://docs.scipy.org/doc/scipy/reference/linalg.html

Here we will look at how to use some of these functions:


In [ ]:
import numpy as np
import scipy.linalg as la
from scipy import Inf

## Linear equation systems

Linear equation systems on the matrix form

$A x = b$

where $A$ is a matrix and $x,b$ are vectors can be solved like:

In [ ]:
A = np.array([[1,2,3], [4,5,6], [7,8,9]])
b = np.array([1,2,3])

In [ ]:
x = la.solve(A, b)
x

In [ ]:
# check
np.dot(A, x) - b

We can also do the same with

$A X = B$

where $A, B, X$ are matrices:

In [ ]:
A = np.random.rand(3,3)
B = np.random.rand(3,3)
vsep='\n'+'-'*30+'\n'
print(A,end=vsep)
print(B)

In [ ]:
X = la.solve(A, B)

In [ ]:
X

In [ ]:
# check
la.norm(np.dot(A, X) - B)

## Eigenvalues and eigenvectors

The eigenvalue problem for a matrix $A$:

$\displaystyle A v_n = \lambda_n v_n$

where $v_n$ is the $n$th eigenvector and $\lambda_n$ is the $n$th eigenvalue.

To calculate eigenvalues of a matrix, use the `eigvals` and for calculating both eigenvalues and eigenvectors, use the function `eig`:

In [ ]:
evals = la.eigvals(A)

In [ ]:
evals

In [ ]:
evals, evecs = la.eig(A)

In [ ]:
evals

In [ ]:
evecs

The eigenvectors corresponding to the $n$th eigenvalue (stored in `evals[n]`) is the $n$th *column* in `evecs`, i.e., `evecs[:,n]`. To verify this, let's try mutiplying eigenvectors with the matrix and compare to the product of the eigenvector and the eigenvalue:

In [ ]:
n = 1
la.norm(np.dot(A, evecs[:,n]) - evals[n] * evecs[:,n])

There are also more specialized eigensolvers, like the `eigh` for Hermitian matrices. Hermitian matrices are matricies with complex elements where the i-th,j-th element is equal to the complex conjugate of the j-th,i-th element, analagous to a symetric matrix for real values.

## Matrix operations

In [ ]:
# the matrix inverse
la.inv(A)

In [ ]:
# determinant
la.det(A)

In [ ]:
# norms of various orders
la.norm(A, ord=2), la.norm(A, ord=Inf)

In [ ]:
# a 2 norm is Euclidean distance of a vector: sqrt(sum of squares of array)
# a 1 norm is city-block distance in a vector: (sum of absolute value array)
# demonstrating a 3-norm on a vector:
x=np.random.normal(0, 10, 100)
n = la.norm(x, ord=3)
n2 = np.sum(np.abs(x**3))**(1 / 3.0)
print('should be ca. zero',  n - n2)

### A matrix algebra example with feed forward/backward chemical reactors

The example is to use matrix algebra tools to solve the a chemical's dispersion 
and decay problem for N tanks or water bodies with flow between them, 
influent and effluent from the system and decay in each reactor.

Solving a mass balance of water and a chemical, we have arrays for each of
the following variables, and the shape of each of those is (number_of_reactors,):

    v = volume of each reactor [L3/T]

    m = chemical_mass in each reactor [M]

    c = concentration of a chemical in each reactor [M/L3]

    k = decay constant [1/T]

    s = external chemical mass sources added to each reactor [M/T]

    dvdt = change in volume in each reactor in time [L3/T]

    dmdt = change in mass of a chemical in each reactor in time [M/T]

Then we have a matrix describing the flow rates between all reactors, so
it's size is (number_of_reactors, number_of_reactors), a connectivity matrix.

    q = flow rate matrix of reactor to reactor [V/T] (constant density here)

Expressions for the derivatives numerically:

    dmdt = dot(q, c) - k * c * v + s     
    
    dvdt = np.sum(q, axis=1)

Update:

    m += dmdt * dt
    
    v += dvdt * dt

    c = m / v

This is sometimes called a continuously stirred tank reactor problem because
concentrations within each reactor are assumed spatially uniform.

In [ ]:
def tank_reactors(v, m, q=0.0, s=0.0, k=0.0, dt=1.0, steps=100, c=None, debug=False):
    if c is None:
        c = m / v 
    for step in range(steps):
        dmdt = np.dot(q, c) - k * c * v + s
        dvdt = np.sum(q, axis=1) 
        v += dvdt * dt
        m += dmdt * dt
        c = m / v
    return v, m, c

If you do not have bokeh installed in your conda environment run
```
% conda install -y bokeh
```

In [ ]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()
num_reactors = 5
v = np.full((num_reactors,), 1.0)
m = np.full((num_reactors,), 1.0)
k = np.full((num_reactors,), 1.0)
q = np.array([[5, -5,  0,  0,  0], # this is feedforward advection
              [5,  0, -5,  0,  0], # this matrix keeps volume constant
              [0,  5,  0, -5,  0],
              [0,  0,  5,  0, -5],
              [0,  0,  0,  5, -5],
             ]) 
s = .1
c = np.array([1, 0, 0, 0, 0])
plot = figure()
integration_steps_per_report, report_steps = 5, 100
concentrations = np.empty((report_steps + 1, num_reactors))
concentrations[0,:] = c
for step in range(report_steps):
    v, m, c = tank_reactors(v, m, q, s, k, dt=0.01, steps=integration_steps_per_report,
                            c=c, debug=False)
    concentrations[step + 1,:] = c
line_colors = ['red', 'blue', 'green', 'black', 'orange']
for lc, idx in zip(line_colors, range(num_reactors)):
    if idx == 0:
        label = "0 upstream"
    elif idx == 4:
        label = "4 downstream"
    else:
        label = str(idx)
    plot.line(range(report_steps + 1), concentrations[:,idx], line_color=lc,
              legend=label)
show(plot)


## Sparse matrices

Sparse matrices are often useful in numerical simulations dealing with large systems, if the problem can be described in matrix form where the matrices or vectors mostly contains zeroes. Scipy has good support for sparse matrices, with basic linear algebra operations (such as equation solving, eigenvalue calculations, etc).

There are many possible strategies for storing sparse matrices in an efficient way. Some of the most common are the so-called coordinate form (COO), list of list (LIL) form, and compressed-sparse column CSC (and row, CSR). Each format has advantanges and disadvantages. Most computational algorithms (equation solving, matrix-matrix multiplication, etc.) can be efficiently implemented using CSR or CSC formats, but they are not so intuitive and not so easy to initialize. Often a sparse matrix is initially created in COO or LIL format (where we can efficiently add elements to the sparse matrix data), and then converted to CSC or CSR before being used in real calculations.

For more information about these sparse formats, see http://en.wikipedia.org/wiki/Sparse_matrix


When we create a sparse matrix, we have to choose which format it should be stored in. For example: 

In [ ]:
import scipy.sparse as sp

In [ ]:
# dense matrix
M = np.array([[1,0,0,0], [0,3,0,0], [0,1,1,0], [1,0,0,1]]); M

In [ ]:
# convert from dense to sparse
A = sp.csr_matrix(M); A

In [ ]:
# convert from sparse to dense
A.todense()

More efficient way to create sparse matrices: create an empty matrix and populate it using matrix indexing (avoids creating a potentially large dense matrix):

In [ ]:
A = sp.lil_matrix((4,4)) # empty 4x4 sparse matrix
A[0,0] = 1
A[1,1] = 3
A[2,2] = A[2,1] = 1
A[3,3] = A[3,0] = 1
A

In [ ]:
A.todense()

Converting between different sparse matrix formats:

In [ ]:
A

In [ ]:
A = sp.csr_matrix(A); A

In [ ]:
A = sp.csc_matrix(A); A

We can compute with sparse matrices like we do with dense matrices:

In [ ]:
A.todense()

In [ ]:
(A * A).todense()

In [ ]:
try:
    np.dot(A, A).todense()
except ValueError as e:
    print(repr(e))
# numpy dot errors out with sparse matricies, but we can use the dot
# method of sparse arrays
A.dot(A).todense()

In [ ]:
v = np.array([1,2,3,4])[:,np.newaxis]; v

In [ ]:
# sparse matrix - dense vector multiplication
A * v

In [ ]:
# same result with dense matrix - dense vector multiplcation
A.todense() * v